In [2]:

import openai
import json
from pykrx import stock
from pykrx import bond

TICKER_SAMSUNG = "005930"

SYSTEM_PROMPT = """
            너는 삼성전자 주식 정보를 알려주는 봇이야. 
            아래와 같은 질문이 온 경우에만 대답해야해
            1. 삼성 또는 삼성전자의 주식 정보
            2. 날짜를 말해야 함
            
            CASE1
            질문이 올바르지 않은 경우 아래와 같은 대답을 출력해줘
            1. 삼성 또는 삼성전자가 아닌 다른 회사의 주식 정보를 물어본 경우
              -> 저는 삼성전자의 주식 정보만 대답할 수 있습니다.
            2. 날짜를 말하지 않은 경우
              -> 날짜의 범위를 지정해서 다시 질문해 주세요.
            3. 그 외 엉뚱한 질문인 경우
              -> 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.

            CASE2
            질문이 올바른 경우 아래와 같은 JSON 형식으로 응답해줘
            {
                start_dt: 사용자가 말한 시작일
                end_dt: 사용자가 말한 종료일
            } 
"""

def is_valid_json(data):
    try:
        json.loads(data)
        return True
    except (TypeError, ValueError):
        return False

def get_market_fundamental(start_dt, end_dt, ticket):
    df = stock.get_market_fundamental(start_dt, end_dt, ticket)
    return df.head(365)

def get_market_ohlcv(start_dt, end_dt, ticket):
    df = stock.get_market_ohlcv(start_dt, end_dt, ticket)
    return df.head(365)

def chat_create(question, prompt):
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=[
            {
                "role": "system",
                "content": prompt
            },
            {
                "role": "user", 
                "content": question
            }],
        )
    return response.choices[0].message.content

def ask(question, prompt, cache):
    if(prompt != ""):
       cache = chat_create(question, prompt)
       return ask(question, "", cache)
    else:
        if not is_valid_json(cache):
            return cache

        ref = json.loads(cache)
        start_dt = ref["start_dt"]
        end_dt = ref["end_dt"]
        fundamental = get_market_fundamental(start_dt, end_dt, TICKER_SAMSUNG)
        ohlcv = get_market_ohlcv(start_dt, end_dt, TICKER_SAMSUNG)

        fundamental_dict = fundamental.to_dict(orient="records")
        ohlcv_dict = ohlcv.to_dict(orient="records")

        fundamental_str = json.dumps(fundamental_dict, ensure_ascii=False)
        ohlcv_str = json.dumps(ohlcv_dict, ensure_ascii=False)

        question = (
            "질문과 데이터를 바탕으로 분석해서 결과를 알려줘"
            + "질문 : " + question
            + ", 데이터1 : " + fundamental_str
            + ", 데이터2 : " + ohlcv_str
        )
        return chat_create(question, "")

def chat():
    print("삼성전자의 주식 정보에 대해 날짜 범위와 함께 질문해 주세요.")
    while True:
        user_input = input("User: ")
        print(f"USER: {user_input}")

        if user_input.lower() == 'exit':
            print("Exiting...")
            break
        response = ask(user_input, SYSTEM_PROMPT, "")
        print(f"GPT: {response}")

if __name__ == "__main__":
    chat()


삼성전자의 주식 정보에 대해 날짜 범위와 함께 질문해 주세요.
USER: 삼성전자 2024년 1월 주식 정보 요약
GPT: 2024년 1월 삼성전자의 주식 정보를 요약해 보겠습니다.

### 1. 재무 지표 요약 (데이터1)
- **BPS (주당 장부가치)**: 50,817 원
- **PER (주가수익비율)**: 9.24 (평균값)
- **PBR (주가순자산비율)**: 1.45 (평균값)
- **EPS (주당순이익)**: 8,057 원
- **DIV (배당수익률)**: 1.88% (평균값)
- **DPS (주당배당금)**: 1,444 원

### 2. 주식 가격 변동 (데이터2)
- **최근 주가 동향**:
    - **최고가**: 79,800 원
    - **최저가**: 70,000 원
    - **종가**: 79,600 원
    - **거래량**: 약 17,142,847 주 (최대 하루 평균)
    - **등락률**: 최근 주가 변동에 따른 평균 등락률은 -0.27%로 보임

### 3. 분석 요약
- **PER 9.24**는 낮은 수치로, 주가가 상대적으로 저평가되고 있음을 나타냅니다.
- 배당수익률이 **1.88%**로 안정적인 수익을 제공할 가능성이 있으며, 성장 전망에 따라 투자 매력이 있을 수 있습니다.
- 주가는 최근 들어 79,600 원으로 안정세를 보였으며, 거래량이 상당히 높아 시장의 활발한 관심을 받고 있음을 알 수 있습니다.
- 주가 등락률이 다소 부정적임에도 불구하고, 장기적으로 안정적인 펀더멘털을 감안했을 때 투자이익을 기대할 수 있습니다.

### 결론
삼성전자는 상대적으로 낮은 PER과 안정된 배당수익률로 매력적인 투자 대상으로 평가됩니다. 다만, 최근 주가 등락이 부정적임에 따라 투자 결정 시 신중해야 할 것입니다.
USER: 
GPT: 무슨 말인지 모르겠어요, 삼성전자의 주식 정보를 날짜 범위를 지정해 질문해 주세요.
USER: 
